https://jira.x5.ru/browse/CVMUPG-1089

In [1]:
name = 'CVMUPG-1089_only_kotex'

In [4]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [5]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

#### Найдем PLU Kotex

In [7]:
plu_hierarchy_lvl_4_dk = ['FD0911001']
# syntethic_category_id = [96,57,75,73,101,55,56,22]
plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [9]:
plu_codes = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [10]:
len(plu_codes)

102

In [11]:
brand_plu = (spark.table("hive_ssa_tc5.cvm_product")
                 .filter(F.col('plu_hierarchy_lvl_4_dk') == 'FD0911001')
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .select('plu_id', 'plu_nm', 'plu_brand_hierarchy_lvl_nm')
                 .distinct()
                 .toPandas())

In [13]:
val = brand_plu.reset_index(drop=True)

In [14]:
val.head()

,plu_id,plu_nm,plu_brand_hierarchy_lvl_nm
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX


### 1: A-24-17-recurrent-Kimberly-clark-brand-w2-no

In [15]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand-w2-no'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65307: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65380 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [16]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [17]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [18]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [19]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [20]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [21]:
val.columns = ['plu_code', 'plu_nm', 'plu_brand_hierarchy_lvl_nm']

In [22]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [23]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN


### 2: A-24-17-recurrent-Kimberly-clark-brand-w2-yes

In [24]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand-w2-yes'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65300: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65365 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [25]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [26]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [27]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [28]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [29]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [30]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [31]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [32]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3: A-24-16-recurrent-Kimberly-clark-all-w2-no

In [33]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all-w2-no'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65289: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65401 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [34]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [35]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [36]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [37]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [38]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [39]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [40]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [41]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4: A-24-16-recurrent-Kimberly-clark-all-w2-yes

In [42]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all-w2-yes'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        65291: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-20',
    'date_communication_to': '2021-10-26',

    'wave_ids': [
        65398 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [43]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [44]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [45]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [46]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

[Stage 352:================================>                   (126 + 74) / 200]22/02/14 17:06:00 ERROR cluster.YarnScheduler: Lost executor 461 on mn-hd0027.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 361:=========>(1195 + 26) / 1221][Stage 362:==========>(3915 + 5) / 3920] 3920]]]]22/02/14 17:15:19 ERROR cluster.YarnScheduler: Lost executor 515 on mn-hd0112.x5.ru: Executor heartbeat timed out after 152162 ms
[Stage 361:=========>(1195 + 22) / 1221][Stage 362:==========>(3915 + 5) / 3920]22/02/14 17:15:19 ERROR cluster.YarnScheduler: Lost executor 529 on mn-hd0112.x5.ru: Executor heartbeat timed out after 153920 ms
[Stage 361:=========>(1195 + 23) / 1221][Stage 362:==========>(3915 + 5) / 3920]22/02/14 17:15:20 ERROR cluster.YarnScheduler: Lost executor 516 on mn-hd0112.x5.ru: Executor heartbeat timed out after 155725 ms
[Stage 361:==========>(1215 + 6) / 1221][

In [47]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [48]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [49]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [50]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 5: X-24-18-KimberlyClark-Kotex-Competitors-2-sof

In [51]:
camp_id = 'X-24-18-KimberlyClark-Kotex-Competitors-2-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63969: "sms_viber_tg",
        63970: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-07',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        64054 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [52]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [53]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [54]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [55]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [57]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [58]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [59]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [60]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 6: X-24-19-KimberlyClark-Kotex-Loyal-2-sof

In [61]:
camp_id = 'X-24-19-KimberlyClark-Kotex-Loyal-2-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63965: "sms_viber_tg",
        63782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-07',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        64055 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [62]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [63]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [64]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [65]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [66]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

22/02/14 19:33:44 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:60332 is closed


In [67]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [68]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [69]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 7: A-24-17-recurrent-Kimberly-clark-brand

In [70]:
camp_id = 'A-24-17-recurrent-Kimberly-clark-brand'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63795: "sms_viber_tg",
        63797: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        65380 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [71]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [72]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [73]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [74]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [75]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

[Stage 730:============================================>       (170 + 30) / 200]44]]]]]]]]22/02/14 20:19:21 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:45984 is closed
[Stage 741:(1325 + 38) / 3920][Stage 742:(1235 + 9) / 3920][Stage 743:(1191 + 30) / 1221]]]22/02/14 20:28:35 ERROR cluster.YarnScheduler: Lost executor 1773 on mn-hd0127.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 745:====================================================>(197 + 3) / 200]22/02/14 20:35:30 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:51356 is closed
22/02/14 20:35:31 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:37240 is closed
22/02/14 20:35:31 ERROR client.TransportResponseHandler: Still hav

In [76]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [77]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [78]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 8: A-24-16-recurrent-Kimberly-clark-all

In [79]:
camp_id = 'A-24-16-recurrent-Kimberly-clark-all'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63794: "sms_viber_tg",
        63796: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63866 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [80]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [81]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [82]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [83]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [84]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [85]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [86]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [87]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 9: X-24-16-KimberlyClark-Kotex-Competitors-sof

In [88]:
camp_id = 'X-24-16-KimberlyClark-Kotex-Competitors-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63780: "sms_viber_tg",
        63785: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63873 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [89]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [90]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [91]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [92]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

[Stage 924:===========>(188 + 12) / 200][Stage 927:===========>(187 + 13) / 200]1) / 1221]]]22/02/14 22:29:24 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:44084 is closed


In [93]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

[Stage 958:(199 + 1) / 200][Stage 961:(138 + 62) / 200][Stage 967:(843 + 1) / 844]]]]]]]]]]22/02/14 22:45:04 ERROR cluster.YarnScheduler: Lost executor 3056 on mn-hd0229.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [94]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [95]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [96]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 10: X-24-15-KimberlyClark-Kotex-Loyal-sof

In [97]:
camp_id = 'X-24-15-KimberlyClark-Kotex-Loyal-sof'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63771: "sms_viber_tg",
        63782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63870 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [98]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [99]:
if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
              .select('customer_rk').distinct()
             )
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
              .select('customer_rk').distinct()
             )

In [100]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [101]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [102]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [103]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [104]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [105]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
val.to_csv('val.csv', index=False)

In [107]:
val = pd.read_csv('val.csv')

### 11: f24_freq24_Kimberly_Clark

In [108]:
camp_id = 'f24_freq24_Kimberly_Clark'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63744: "sms_viber_tg",
        63744: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        -1 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [109]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [110]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [111]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [112]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [113]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [114]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [115]:
val['plu_code'] = val['plu_code'].astype('str')

In [116]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [117]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x.28,qty_y.28,qty_x.29,qty_y.29,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 12: U-24-12-kotex-loyal-viber-sms

In [118]:
camp_id = 'U-24-12-kotex-loyal-viber-sms'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63779: "sms_viber_tg",
        63783: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63865 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [119]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [120]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [121]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [122]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

[Stage 1228:=========>(3919 + 1) / 3920][Stage 1229:=========>(1220 + 1) / 1221]2) / 200]]]]]22/02/15 01:09:37 ERROR client.TransportClient: Failed to send RPC RPC 4683526459233114546 to /192.168.234.58:44752: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/15 01:09:37 ERROR client.TransportClient: Failed to send RPC RPC 5073032172408701441 to /192.168.234.58:36792: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/15 01:09:37 ERROR client.TransportClient: Failed to send RPC RPC 8808844225295857600 to /192.168.234.58:41732: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/15 01:09:37 ERROR client.TransportClient: Failed to send RPC RPC 50

In [123]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

In [124]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

[Stage 1297:(3883 + 37) / 3920][Stage 1299:(1211 + 10) / 1221][Stage 1301:(204 + 6) / 210]]22/02/15 01:41:52 ERROR server.TransportChannelHandler: Connection to /192.168.234.58:44126 has been quiet for 120000 ms while there are outstanding requests. Assuming connection is dead; please adjust spark.network.timeout if this is wrong.
22/02/15 01:41:52 ERROR server.TransportChannelHandler: Connection to /192.168.234.58:44124 has been quiet for 120000 ms while there are outstanding requests. Assuming connection is dead; please adjust spark.network.timeout if this is wrong.
22/02/15 01:41:52 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:44124 is closed
22/02/15 01:41:52 ERROR spark.ContextCleaner: Error cleaning broadcast 2180
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [120 seconds]. This timeout is controlled by spark.rpc.askTimeout
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$crea

In [125]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [126]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 13: U-24-12-kotex-competitors-viber-sms

In [127]:
camp_id = 'U-24-12-kotex-competitors-viber-sms'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        63760: "sms_viber_tg",
        63769: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-10-06',
    'date_communication_to': '2021-10-17',

    'wave_ids': [
        63865 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [128]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [129]:
if len(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())) > 1:
    if str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) != str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]):
        guests_tg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
                  .select('customer_rk').distinct()
                 )

        guests_cg = (experiment_guests_communications
                  .filter(F.col('group_name') == '0_' +
                        str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[1]) + '_' +
                        list(experiment_params.map_camp_wave_ids_to_suffixes.values())[1])
                  .select('customer_rk').distinct()
                 )
    else:
        print("Error - check the camp waves ids")
else:
    guests_tg = (experiment_guests_communications
              .filter(F.col('group_name') == '0_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

    guests_cg = (experiment_guests_communications
              .filter(F.col('group_name') == '1_' +
                    str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                    list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
              .select('customer_rk').distinct()
             )

In [130]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)
start_date_pre = start_date - datetime.timedelta(days=11)
end_date_pre = start_date - datetime.timedelta(days=1)
start_date_post = end_date + datetime.timedelta(days=1)
end_date_post = end_date + datetime.timedelta(days=11)

In [131]:
seg_pd_period_tg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_period_cg = get_segment_grouped_one_plu(spark, 
                    start_date, 
                    end_date, 
                    plu_codes,
                    guests=guests_cg)

In [132]:
seg_pd_pre_tg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_pre_cg = get_segment_grouped_one_plu(spark, 
                    start_date_pre, 
                    end_date_pre, 
                    plu_codes,
                    guests=guests_cg)

22/02/15 02:34:03 ERROR client.TransportResponseHandler: Still have 6 requests outstanding when connection from /192.168.234.58:44774 is closed


In [133]:
seg_pd_post_tg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_tg)
seg_pd_post_cg = get_segment_grouped_one_plu(spark, 
                    start_date_post, 
                    end_date_post, 
                    plu_codes,
                    guests=guests_cg)

In [134]:
val = pd.merge(val, seg_pd_period_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_period_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_pre_cg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_tg, how='left', on=['plu_code', 'plu_nm'])
val = pd.merge(val, seg_pd_post_cg, how='left', on=['plu_code', 'plu_nm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'qty_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [135]:
val.head()

,plu_code,plu_nm,plu_brand_hierarchy_lvl_nm,qty_x,qty_y,qty_x.1,qty_y.1,qty_x.2,qty_y.2,qty_x.3,...,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y,qty_x,qty_y
0,4133996,Котекс прокладки ультра сетч ночные 24шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17580,KOTEX Тампоны МИНИ 16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58458,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4148799,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2125858,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
list_columns = ['plu_code',
'plu_nm', 'plu_brand_hierarchy_lvl_nm',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-no',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-17-recurrent-Kimberly-clark-brand-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-no',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'A-24-16-recurrent-Kimberly-clark-all-w2-yes',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-18-KimberlyClark-Kotex-Competitors-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'X-24-19-KimberlyClark-Kotex-Loyal-2-sof',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-17-recurrent-Kimberly-clark-brand',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'A-24-16-recurrent-Kimberly-clark-all',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-16-KimberlyClark-Kotex-Competitors-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'X-24-15-KimberlyClark-Kotex-Loyal-sof',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'f24_freq24_Kimberly_Clark',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-loyal-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms',
'U-24-12-kotex-competitors-viber-sms']

In [137]:
for i in range(3, 79, 6):
    list_columns[i] += '_period_tg'
    list_columns[i+1] += '_period_cg'
    list_columns[i+2] += '_pre_tg'
    list_columns[i+3] += '_pre_cg'
    list_columns[i+4] += '_post_tg'
    list_columns[i+5] += '_post_cg'

In [138]:
val.columns = list_columns

In [139]:
val.iloc[:, 3:] = val.iloc[:, 3:].astype('float')

In [140]:
val = val.T

In [141]:
val

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
plu_code,4133996,17580,58458,4148799,2125858,14762,3943538,3329057,4048146,96637,...,3202724,70574,2041665,3954340,3954339,2069986,4133997,3017396,3062987,2125862
plu_nm,Котекс прокладки ультра сетч ночные 24шт,KOTEX Тампоны МИНИ 16шт,KOTEX Прокладки DEO супертонк.ежед.20шт,KOTEX Тампоны НЭЧУРАЛ СУПЕР жен.гиг.16шт,KOTEX Прокладки NORMAL ежедневные 45шт,KOTEX BREV.Прокл.дыш.на к/д20шт,KOTEX Прокл.AKT.DEO Э/ТОН.ежед.гиг.48шт,KOTEX Прокладки EXTRA SOFT SUPER гиг.8шт,Прок. Kotex Ul нормал 40 шт,KOTEX BREV.Прокл.LONGдл.к/д18шт,...,KOTEX Прокладки СТРИНГ ежедневные 40шт,KOTEX Пакеты NORMAL каж.д. 20шт,KOTEX Тампоны МИНИ 8шт,KOTEX Прокл.AKT.DEO Э/ТОН.ежед.гиг.16шт,KOTEX Прокл.ACT.СУП.ПЛ.жен.гиг.крыл.7шт,KOTEX Тампоны НОРМАЛ 32шт,Котекс трусики ночные 2шт,KOT.Прокл.УЛ.ДР-СОФТ СУП.Д.16шт,Прокладки Kotex ультра Драй&Соф,KOTEX Прокладки DEO ежедневные 45шт
plu_brand_hierarchy_lvl_nm,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,...,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX,KOTEX
A-24-17-recurrent-Kimberly-clark-brand-w2-no_period_tg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_period_cg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_pre_tg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_pre_cg,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_post_tg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-no_post_cg,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A-24-17-recurrent-Kimberly-clark-brand-w2-yes_period_tg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
val.to_excel('only_kotex_1' +'.xlsx')

In [143]:
val = val.T

In [144]:
val.to_excel('only_kotex_2' +'.xlsx')

22/02/15 03:48:14 ERROR client.TransportClient: Failed to send RPC RPC 7106114592155545591 to /192.168.234.58:58718: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/15 03:48:14 ERROR client.TransportClient: Failed to send RPC RPC 5785113522903576357 to /192.168.234.58:58718: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/15 03:48:32 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/02/15 03:48:32 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Attempt recovered after RM restartDue to executor failures all available nodes are blacklisted
22/02/15 03:48:32 ERROR client.TransportClient: Failed to send RPC RPC 8928417379027134843 to /192